# Medicare Part D Data Cleaning and Feature Engineering
Data Source: Data Source: https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/yvpj-pmj2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

In [2]:
# setting styles for plotting
plt.rcParams['figure.figsize'] = [20, 5]
plt.rcParams['image.cmap'] = 'bone'
sns.set_palette('bone')
sns.set_style('whitegrid')
sns.set_context('poster')

In [3]:
# read in medicare state drug utilization data
drugs2016 = pd.read_csv('Medicare_Provider_Utilization_and_Payment_Data__2016_Part_D_Prescriber.csv')
print(drugs2016.shape)
drugs2016.head()

(24964300, 21)


,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65
0,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,ATORVASTATIN CALCIUM,ATORVASTATIN CALCIUM,NaN,...,15.0,450,139.32,NaN,*,13.0,NaN,15.0,450.0,139.32
1,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,CIPROFLOXACIN HCL,CIPROFLOXACIN HCL,NaN,...,11.0,96,80.99,NaN,*,NaN,*,NaN,NaN,NaN
2,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,DOXYCYCLINE HYCLATE,DOXYCYCLINE HYCLATE,20.0,...,20.0,199,586.12,NaN,#,NaN,#,NaN,NaN,NaN
3,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,ELIQUIS,APIXABAN,NaN,...,17.0,510,6065.02,NaN,*,17.0,NaN,17.0,510.0,6065.02
4,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,FUROSEMIDE,FUROSEMIDE,12.0,...,17.0,405,45.76,NaN,#,NaN,#,NaN,NaN,NaN


In [4]:
drugs2016.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_city', 'nppes_provider_state', 'specialty_description',
       'description_flag', 'drug_name', 'generic_name', 'bene_count',
       'total_claim_count', 'total_30_day_fill_count', 'total_day_supply',
       'total_drug_cost', 'bene_count_ge65', 'bene_count_ge65_suppress_flag',
       'total_claim_count_ge65', 'ge65_suppress_flag',
       'total_30_day_fill_count_ge65', 'total_day_supply_ge65',
       'total_drug_cost_ge65'],
      dtype='object')

In [5]:
# read in drug info
druglist = pd.read_excel('20180208_HEDIS 2018_NDC_MLD_Directory_Complete_Workbook.xlsx',
                         'Medications List to NDC Codes')
druglist.head()

,Medication List,NDC Code,Brand Name,Generic Product Name,Route,Description,Drug ID,Drug Name,Package Size,Unit,Dose,Form,MED Conversion Factor
0,5-ARI Medications,54039513,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
1,5-ARI Medications,54039522,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
2,5-ARI Medications,93565556,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
3,5-ARI Medications,93565598,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN
4,5-ARI Medications,115143808,Dutasteride,dutasteride 0.5 mg oral capsule,oral,5-ARI Medications,d04788,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# new dataframe with only opioid drugs
# we can tell if a drug is an opioid if the MED (Morphine Equivalent Dose) is not nan
opioiddrugs = druglist[druglist['MED Conversion Factor'].notnull()].copy()
opioiddrugs['Drug Name'] = opioiddrugs['Drug Name'].str.upper()
opioiddrugs.head()

,Medication List,NDC Code,Brand Name,Generic Product Name,Route,Description,Drug ID,Drug Name,Package Size,Unit,Dose,Form,MED Conversion Factor
52293,Opioid Medications,591264101,APAP/Butalbital/Caffeine/Codeine,acetaminophen/butalbital/caffeine/codeine 300 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52294,Opioid Medications,52544008201,Fioricet with Codeine,acetaminophen/butalbital/caffeine/codeine 300 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52295,Opioid Medications,143300001,APAP/Butalbital/Caffeine/Codeine,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52296,Opioid Medications,247084530,Fioricet with Codeine,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15
52297,Opioid Medications,591322001,APAP/Butalbital/Caffeine/Codeine,acetaminophen/butalbital/caffeine/codeine 325 ...,oral,Opioid Medications,d03425,CODEINE,NaN,mg,30.0,NaN,0.15


In [7]:
opioiddrugs['Drug Name'].unique()

array(['CODEINE', 'DIHYDROCODEINE', 'HYDROCODONE', 'OXYCODONE',
       'PENTAZOCINE', 'TRAMADOL', 'OPIUM', 'BUPRENORPHINE', 'BUTORPHANOL',
       'FENTANYL', 'HYDROMORPHONE', 'LEVORPHANOL', 'MEPERIDINE',
       'METHADONE', 'MORPHINE', 'MORPHINE EQUIVALENT', 'OXYMORPHONE',
       'TAPENTADOL'], dtype=object)

In [8]:
## convert drug name column to list
opioids = opioiddrugs['Drug Name'].unique().tolist()
## join list of strings to create regex
opioids = '|'.join(opioids)
## use str.contains to match pattern
opioids_ = drugs2016[(drugs2016['drug_name'].str.contains(opioids)) | (drugs2016['generic_name'].str.contains(opioids))]

In [9]:
opioids

'CODEINE|DIHYDROCODEINE|HYDROCODONE|OXYCODONE|PENTAZOCINE|TRAMADOL|OPIUM|BUPRENORPHINE|BUTORPHANOL|FENTANYL|HYDROMORPHONE|LEVORPHANOL|MEPERIDINE|METHADONE|MORPHINE|MORPHINE EQUIVALENT|OXYMORPHONE|TAPENTADOL'

In [10]:
def make_opioid_df(df):
    df = df[df['generic_name'].str.contains(opioids)].copy()
    return df

In [11]:
opioiddrugs2016 = make_opioid_df(drugs2016)

In [12]:
## there are asthma meds here!?
opioiddrugs2016.drug_name.unique()

array(['ACETAMINOPHEN-CODEINE', 'BUTRANS', 'FENTANYL',
       'HYDROCODONE-ACETAMINOPHEN', 'MORPHINE SULFATE ER',
       'OXYCODONE HCL', 'OXYCODONE-ACETAMINOPHEN', 'OXYCONTIN',
       'TRAMADOL HCL', 'TRAMADOL HCL-ACETAMINOPHEN', 'MEPERIDINE HCL',
       'SPIRIVA', 'METHADONE HCL', 'COMBIVENT RESPIMAT', 'MS CONTIN',
       'VICODIN', 'LEVORPHANOL TARTRATE', 'IPRATROPIUM-ALBUTEROL',
       'TRAMADOL HCL ER', 'SUBOXONE', 'STIOLTO RESPIMAT',
       'HYDROMORPHONE HCL', 'SPIRIVA RESPIMAT', 'OXYMORPHONE HCL ER',
       'NUCYNTA ER', 'IPRATROPIUM BROMIDE', 'OPANA ER',
       'MORPHINE SULFATE', 'OXYCODONE HCL ER', 'PENTAZOCINE-NALOXONE HCL',
       'EMBEDA', 'BUTORPHANOL TARTRATE', 'BUTALBITAL COMPOUND-CODEINE',
       'DILAUDID', 'ULTRAM', 'ATROVENT HFA', 'BUPRENORPHINE HCL',
       'CODEINE SULFATE', 'ENDOCET', 'KADIAN', 'OXYMORPHONE HCL',
       'SUBSYS', 'ASCOMP WITH CODEINE', 'NUCYNTA', 'HYSINGLA ER',
       'BUPRENORPHINE-NALOXONE', 'HYDROCODONE-IBUPROFEN',
       'HYDROMORPHONE ER', 

In [13]:
# removing asthma drugs
asthmadrugs = ['COMBIVENT RESPIMAT', 'SPIRIVA RESPIMAT', 'ATROVENT', 'STIOLTO RESPIMAT', 'IPRATROPIUM BROMIDE', 'SPIRIVA', 'IPRATROPIUM-ALBUTEROL']

opioiddrugs2016 = opioiddrugs2016[~opioiddrugs2016.drug_name.isin(asthmadrugs)]

In [14]:
opioiddrugs2016.drug_name.unique()

array(['ACETAMINOPHEN-CODEINE', 'BUTRANS', 'FENTANYL',
       'HYDROCODONE-ACETAMINOPHEN', 'MORPHINE SULFATE ER',
       'OXYCODONE HCL', 'OXYCODONE-ACETAMINOPHEN', 'OXYCONTIN',
       'TRAMADOL HCL', 'TRAMADOL HCL-ACETAMINOPHEN', 'MEPERIDINE HCL',
       'METHADONE HCL', 'MS CONTIN', 'VICODIN', 'LEVORPHANOL TARTRATE',
       'TRAMADOL HCL ER', 'SUBOXONE', 'HYDROMORPHONE HCL',
       'OXYMORPHONE HCL ER', 'NUCYNTA ER', 'OPANA ER', 'MORPHINE SULFATE',
       'OXYCODONE HCL ER', 'PENTAZOCINE-NALOXONE HCL', 'EMBEDA',
       'BUTORPHANOL TARTRATE', 'BUTALBITAL COMPOUND-CODEINE', 'DILAUDID',
       'ULTRAM', 'ATROVENT HFA', 'BUPRENORPHINE HCL', 'CODEINE SULFATE',
       'ENDOCET', 'KADIAN', 'OXYMORPHONE HCL', 'SUBSYS',
       'ASCOMP WITH CODEINE', 'NUCYNTA', 'HYSINGLA ER',
       'BUPRENORPHINE-NALOXONE', 'HYDROCODONE-IBUPROFEN',
       'HYDROMORPHONE ER', 'DURAGESIC', 'ROXICODONE', 'ZOHYDRO ER',
       'ZUBSOLV', 'EXALGO', 'VICODIN HP',
       'BUTALB-CAFF-ACETAMINOPH-CODEIN', 'PERCOCET',

In [15]:
opioiddrugs2016['year'] = 2016

In [17]:
opioiddrugs2016.sample(5)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
11089269,1972667434,HERLIHY,TIMOTHY,TOWSON,MD,Internal Medicine,S,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,NaN,...,1053,499.67,NaN,#,NaN,#,NaN,NaN,NaN,2016
7377617,1073543427,MAHLER,EILEEN,CHICAGO,IL,Nurse Practitioner,S,OXYCODONE-ACETAMINOPHEN,OXYCODONE HCL/ACETAMINOPHEN,42.0,...,3123,12967.27,16.0,NaN,35.0,NaN,35.0,984.0,4172.85,2016
10301566,1154399160,ORTIZ-CAMACHO,HERNAN,DORADO,PR,Family Practice,S,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,20.0,...,222,188.20,NaN,#,NaN,#,NaN,NaN,NaN,2016
23192173,1790892768,BREY,ROBIN,SAN ANTONIO,TX,Neurology,S,TRAMADOL HCL,TRAMADOL HCL,NaN,...,796,312.61,NaN,*,NaN,*,NaN,NaN,NaN,2016
17665901,1790703734,JUNTUNEN,KRISTINE,APPLETON,WI,Internal Medicine,S,OXYCODONE HCL,OXYCODONE HCL,12.0,...,529,706.78,NaN,*,NaN,*,NaN,NaN,NaN,2016


In [18]:
# saving cleaned dataframe to csv
opioiddrugs2016.to_csv('medicare_d_2016_opioids.csv')